## Specialization Generator
Generates synonyms for specialization entity

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

import numpy as np
import pandas as pd
import re

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [18]:
def syn_specialization(spec):
    spec = str(spec)
    specs = []
    temp = [None]*3
#     specy = re.sub('\)', ']', spec)
    tspec = re.sub('\]', '', spec)
#     specs.append(re.sub('\(', '[', specy))
    specs.append(re.sub('\[', '- ', tspec))
    specs.append(re.sub('\[', '', tspec))
    specs = list(set(specs + [spec]))
    specs = specs+temp
    return specs[:3]

def con_brk(st):
    st = str(st)
    st = st.replace('[','(')
    return st.replace(']',')')
syn_specialization('diploma in dialysis technology -  dd')

['diploma in dialysis technology -  dd', None, None]

In [4]:
data = pd.read_csv('../data/all_institutions_course_ids_live_new.csv', encoding='latin1').astype(str)

In [5]:
data['spec'] = data['discipline'].copy()
data = data.loc[:,['discipline', 'spec', 'discipline_group', 'programme']]

In [6]:
data.columns

Index(['discipline', 'spec', 'discipline_group', 'programme'], dtype='object')

In [7]:
data.head(3)

,discipline,spec,discipline_group,programme
0,Computer Science,Computer Science,Computer Application,M.C.A. -Master of Computer Applications
1,Master of Computer Application,Master of Computer Application,Computer Application,M.C.A. -Master of Computer Applications
2,COMPUTER APPLICATION,COMPUTER APPLICATION,Computer Application,M.C.A. -Master of Computer Applications


In [8]:
data['spec'].unique().shape

(33007,)

In [9]:
data['spec'].str.lower().unique().shape

(28535,)

In [10]:
data.spec = data.spec.str.lower()

In [11]:
def remove_dot(x):
    try:
        s = ''
        l = re.split('\.|\-|\_|,|\s', x.strip())
        for i in range(len(l)-1):
            if len(l[i]) < 3 and len(l[i+1]) < 3: s = s+l[i]+'.'
            else: s = s+l[i]+' '
        ans = s+' '+l[-1] if len(l[-1])>3 else s+l[-1]
        return re.subn('\.', '', ans)[0].strip()
    except:
        return x.strip()
remove_dot(' master b.sc.engg-el.e.ct.r.ic.al eng. ')

In [12]:
data.spec = data['spec'].apply(remove_dot)
data.spec.unique().shape[0]

26473

In [13]:
def full(s, l):
    for elem in l:
        if ''.join([w[0] for w in elem.split()]) == s: return elem
    else: return 'False'
full('mba', ['mba', 'master bussiness admin', 'mab', 'mas buss adm'])

'master bussiness admin'

In [42]:
data['spec'] = data['spec'].apply(con_brk)

In [43]:
def remo_brace(s):
    s = re.sub(r'\b(\w+)\slanguage$', '\1', s)
    s = re.subn(r'\b(mysore|asas|kochi|ase|amr|mkt|hsp|bp|sf|lko|u\s?g|:|se|any|all)\b', '', s)[0]
    s = re.sub(r'\b(b|m)\s(tech|ba|ca|sc|com)\b', r'\1\2', s)
    s = re.subn('\s+', ' ', s)[0]
    s = re.sub('mgmt', 'management', s)
    s = re.sub('\s*engg\S*', ' engineering ', s)
    s = re.sub('\s*agri\S+', ' agriculture ', s)
    s = re.sub(r'\bsc(\.?\S+|\b)\s*', ' science ', s)
    s = re.sub('\s+of |\s+in ',' ',s) 
    s = re.subn('\s+', ' ', s)[0]
    s = re.subn('\.', '', s)[0]
    s = re.sub('\s?&|\+\s?', ' and ', s)
    s = re.sub('comp\S*', 'computer',s)
    s = re.sub('com\S*\s+app\S*', ' computer application ', s)    
    s = re.sub('_|-', ' ', s)
    s = re.subn('\s+', ' ', s)[0]
    s = re.split('\s*[\(\)<>]+\s*', s)
    s = list(map(lambda x: re.sub('full-time\s?$|ft\s?|reg\S+\s?$|full time', '', x), s))
    s = list(map(lambda x: re.sub('^le$|^lateral$|^leet$|^2nd (year|yr) direct$', 'lateral entry', x), s))
    for part in s:
        while full(part, s) in s:
            s.remove(full(part, s))
    s = ' '.join(s).strip()
    s = re.subn('\s+', ' ', s)[0]
    s = re.sub('\s*(lateral entry|full time)\s+(.*$)', r' \2 \1 ', s).strip()
    nums = {'one':1, 'two':2, 'three':3, 'four':4}
    s = re.sub('(one|two|three|four)', lambda m: str(nums[m.group(1)]), s)
    s = re.sub('\s+yrs?\s+', ' year ', s)
    s = re.sub('\s+arts?\s+', ' art ', s)
    s = re.sub('i?t and computer', 'it and computer', s)
    return s
            
remo_brace('mba(le)(master: of   business admisnistration)  (mas bus adm )one yr   (computer sc.45&engg)')

'mba 1 year computer science engineering lateral entry'

In [44]:
data['remo_br'] = data.spec.apply(remo_brace)
data.remo_br.unique().shape[0]

22651

In [52]:
degrees = list(data.programme.apply(lambda x: x.split('-')[0]))
degrees.extend(data.programme.apply(lambda x: x.split('-')[-1]))
degrees = set(degrees)
deg = []
for d in degrees:
    d = d.replace('.','')
    d = d.lower()   
    d = d.replace('cations', 'cation')
    d = d.replace('&', 'and')
    d = d.replace(' in ', ' ')
    d = d.replace(' of ', ' ')
    d = d.replace('(', ' ')
    d = d.replace(')', ' ')
    d = re.sub('\s+', ' ', d)
    d = d.split(',')
    deg.extend([de.strip() for de in d])
degrees = deg
degrees.sort()
set(degrees)

{'acharya',
 'alankar',
 'anm',
 'anu parangat',
 'auxiliary nurse and midwife',
 'ayurveda vachaspati',
 'ayurvedacharya',
 'ba',
 'ba bed',
 'ba hons',
 'ba llb',
 'bachelor agriculture',
 'bachelor architecture',
 'bachelor arts',
 'bachelor arts honors',
 'bachelor audiology and speech language pathology',
 'bachelor ayurved medicine',
 'bachelor ayurved medicine and surgery',
 'bachelor ayurved pharmacy',
 'bachelor business administration',
 'bachelor business management',
 'bachelor business studies',
 'bachelor chemical engineering',
 'bachelor chemical technology',
 'bachelor civil engineering',
 'bachelor civil law',
 'bachelor clinical optometry',
 'bachelor commerce',
 'bachelor computer application',
 'bachelor dance',
 'bachelor dental surgery',
 'bachelor design',
 'bachelor education',
 'bachelor engineering',
 'bachelor fashion technology',
 'bachelor fine arts',
 'bachelor fisheries science',
 'bachelor general law',
 'bachelor homeopathic medicine and surgery',
 'bac

In [54]:
patt = r'\b('+'|'.join(list(set(degrees)))+r')\b'
def rm_deg(s):
    s = re.sub(patt, ' ',s)
    s = re.sub('\s+', ' ', s)
    return s.strip()

rm_deg('mba mbadd damba master computer application ma mba')

\b(bvsc|chemistry|graduate programme management|bms|bdes|bchemtech|bachelor ayurved medicine|pgdm|bachelor general law|mcom bed|master journalism and mass communication|bachelor chemical engineering|mfa|bachelor naturopathy and yogic sciences|anu parangat|bachelor journalism|mlibsc|master homeopathic medicine and science|ma bed|mot|master engineering|bsms|doctor music|bhtm|pharmd|dmus|master fishery science|blibisc|bachelor management studies|bfa|bgl|certified public|master business administration|msc nursing|bl|master dental surgery|bachelor hotel and tourism management|bachelor performing arts|mba tech|master technology|doctor philosophy|bachelor unani medicine and surgery|bsc hons|mds|bachelor occupational therapy|master arts|mcom|bhmtt|bachelor arts honors|vidya vachaspati|integrated msc|lld|bachelor law|bachelor arts|master urban planning|mmc|bachelor technology|integrated master computer application|master music|mfm|ms|master planning|master veterinary sciences|bftech|mdance|bach

'mbadd damba'

In [55]:
data['remo_deg'] = data.remo_br.apply(rm_deg)
data.remo_deg.unique().shape[0]

20538

In [56]:
list(data.remo_deg.unique())

['computer science',
 '',
 'computer application',
 'ca',
 'integrated computer science',
 'computer and informatics',
 'computer',
 'computer science and applications',
 'department',
 'masters computer application',
 'application',
 'information technology',
 'science',
 'management',
 'it',
 'informatics',
 'mca1',
 'lateral',
 'commerce',
 'it and computer',
 'information security management services',
 'engineering and technology',
 'infrastructure management services init',
 'storage and cloud technology',
 'mcm',
 'subjects',
 'lateral entry',
 'lmca',
 'computer application cation',
 'technical',
 'c c and and java rdbms computer security computer ne2rk',
 'computer technology',
 '10',
 'commerce and management',
 'information',
 'computer application lateral entry',
 'mc01',
 'information and technology',
 'cs',
 'computer science and application',
 '3yrs / 3rd year 3rd year course',
 '5yrs',
 'pg',
 'post graduate',
 'cdsnw java os ada',
 'pgdca',
 'general',
 'discipline',
 

In [57]:
def clean(s):
    s = re.sub(r'\band$', '', s)
    s.replace('/', ' ')
    s = re.sub(r'^.{,2}$', '', s)
    s = re.sub(r'^\d+$', '', s)
    s = re.sub(r'\bmath\w+\b', 'mathematics', s)
    s = re.sub('\s+', ' ', s)
    return s

In [59]:
data['clean'] = data.remo_deg.apply(clean)
data.clean.unique().shape[0]

20175

In [74]:
data[data['clean_final'] == 'nil'].shape

(18, 9)

In [63]:
data['clean_final'] = np.logical_or(data['clean'], data['discipline_group'].str.lower())

In [76]:
data.loc[np.logical_or(data['discipline']=='nan', data['discipline']=='nil'),'clean_final']  = data.loc[np.logical_or(data['discipline']=='nan', data['discipline']=='nil'),'discipline_group'].str.lower()

## Time to check our final result

In [81]:
data.clean_final.unique().shape[0]

20195

In [79]:
data.spec = data.clean_final

In [91]:
data.spec.unique()

array(['computer science', 'computer application',
       'integrated computer science', ..., 'pgdemm',
       'media studies : film and television',
       'public relations and event management'], dtype=object)

In [95]:
data['discipline_clean'] = data.spec

In [98]:
data.drop_duplicates(['discipline'])[['discipline', 'discipline_clean']].to_csv('cleaned_discipline.csv' )

In [100]:
data.to_csv('cleaned_discipline_full.csv')

In [108]:
spec = data.drop_duplicates(['discipline'])[['discipline_clean', 'discipline']]

In [109]:
spec.shape

(33007, 2)

In [112]:
spec['old_id'] = ['uc_d'+str(i+100000) for i in range(spec.shape[0])]

In [119]:
from collections import defaultdict
d = {discipline:'c'+str(i+100000) for i, discipline in enumerate(spec.discipline_clean.unique())}
len(d)

20185

In [125]:
spec['cleaned_id'] = spec['discipline_clean'].apply(lambda x: d[x])
spec.head()

,discipline_clean,discipline,old_id,cleaned_id
0,computer science,Computer Science,uc_d100000,c100000
1,computer application,Master of Computer Application,uc_d100001,c100001
2,computer application,COMPUTER APPLICATION,uc_d100002,c100001
3,computer application,CA,uc_d100003,c100001
4,integrated computer science,Integrated Computer Science,uc_d100004,c100002


In [126]:
spec.to_csv('clean_unclean_discipline_mapping.csv', header=None, index=None)

In [128]:
spec.drop_duplicates(['cleaned_id'])[['discipline_clean', 'cleaned_id']].to_csv('discipline_dialogflow.csv', index=None, header=None)


# End